# Table of contents
* [How to Run GUI Applications in a Docker Container](#how-to-run-gui-applications-in-a-docker-container)
    * [Forwarding An X Socket to A Docker Container](#forwarding_an_x_socket_to_a_cocker_container)
    * [Handling X Authentication](#handling-x-authentication)



# [How to Run GUI Applications in a Docker Container](https://www.howtogeek.com/devops/how-to-run-gui-applications-in-a-docker-container/)
Docker's normally used to containerise background applications and CLI programs. You can also use it to run graphical programs though! You can either use an existing X Server, where the host machine is already running a graphical environment, or you can run a VNC server within the container.

First it's important to understand what Docker actually does. A Docker "container" is a form of *encapsulation* which seems to be superficially similar to a virtual machine. Unlike a virtual machine, containers share the same Linux kernel as their host system.

The next component is the X Window System. <font color=green>**X Servers such as Xorg provide the fundamental graphical capabilities of Unix systems. GUI applications can't render without an X Server available**</font>. (Alternative windowing systems, such as Wayland, are available - we're focusing on X in this article.)

Trying to run an X Server in Docker is theoretically possible but rarely used. You'd need to run Docker in privileged mode
```shell
 --privileged
```
so it could access your host's hardware. Starting the server would try to claim your video devices, usually resulting in loss of video output as your host's original X server gets its devices yanked away.

<font color='green'>**A better approach is to mount your host's X Server socket into the Docker container**</font>. This allows your container to use the X Server you already have. GUI applications running in the container would then appear on your existing desktop.

## Forwarding An X Socket to A Docker Container

Providing a Docker container with access to your host's X socket is a straightforward procedure. The X socket can be found in
```shell
/tmp/.X11-unix
```
on your host. The contents of this directory should be mounted into a Docker volume assigned to the container. You'll need to use the `host` networking mode for this to work.

You must also provide the container with a `DISPLAY` environment variable. This instructs X clients - your graphical programs - which X server to connect to. Set `DISPLAY` in the container to the value of `$DISPLAY` on your host.

You can encapsulate all this configuration in one `docker-compose.yml` file:
```txt
version: "3"
services:
app:
    image: my-app:latest
    build: .
    environment:
    - DISPLAY=${DISPLAY}
    volumes:
    - /tmp/.X11-unix:/tmp/.X11-unix
    network_mode: host
```

This approach should only be used when you trust your Docker container. Exposing the host's display server is a security risk if you're not completely sure what lies inside the container.


## Handling X Authentication

You might need to authenticate the container to access the X Server. First get an X authentication token from your host machine. Run `xauth list` and note down one of the listed cookies. You'll need to copy the entire line.

![xauth_list](https://static1.howtogeekimages.com/wordpress/wp-content/uploads/csit/2021/04/4a47a0db.png?q=50&fit=crop&w=943&dpr=1.5)

Inside the Docker container, install the xauth package. Then run `xauth add`, passing the token you copied in the previous step.
```shell
apt install -y xauth
xauth add <token>
```
Your container should now successfully authenticate to the X Server.